## Importing Python modules

In [1]:
from twarc import Twarc2, expansions
import os
import pandas as pd
import requests
import time

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Input your Twitter Bearer Token

In [2]:
TOKEN = "YOUR TWITTER ACADEMIC TRACK BEARER TOKEN"

## Query Parameter (e.g., search keyword, time)

In [3]:
os.environ["BEARER_TOKEN"] = TOKEN
bearer_token = os.environ["BEARER_TOKEN"]

def auth():
    return os.environ.get("BEARER_TOKEN")

next_token = ""
def create_url():
    if next_token == "":
        return "https://api.twitter.com/2/tweets/search/all"
    else: 
        return "https://api.twitter.com/2/tweets/search/all?next_token={}".format(next_token)
    
search_url = create_url()
    
query_params = {'query': "(PRSA OR @PRSA OR #PRSA) lang:en", # search query
                'tweet.fields': "author_id,created_at,public_metrics,in_reply_to_user_id",
                'start_time': "2022-01-01T00:00:00Z", # data collection start time
                'end_time': '2022-01-03T00:00:00Z',   # data collection end time
                'max_results': 300,
                "expansions":"referenced_tweets.id"}

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

## Crawling Start

In [4]:
df = pd.DataFrame() 
retweet_df_disk = pd.DataFrame() 

next_token = ''
full_text = ''
row = ''

count = 0

full_text_column = []
includes_id = []
includes_retweet = []

while True:
    bearer_token = auth()
    search_url = create_url()
    headers = create_headers(bearer_token)    
    json_response = connect_to_endpoint(search_url, headers, query_params)
    
    tweet = pd.DataFrame.from_dict(json_response['data']) 
    try:
        tweet_1 = tweet.explode('referenced_tweets')
        tweet_1 = tweet_1.referenced_tweets.apply(pd.Series)
        tweet_2 = tweet_1[["id", "type"]]
        tweet_2.columns = ['retweet_id', 'type']
        tweets = pd.concat([tweet, tweet_2], axis=1)
    
        ### data file
        df = pd.concat([df, tweets], axis=1)

        result_count = json_response['meta']['result_count']
        count += result_count
        print(count)
        time.sleep(3.1)

    except KeyError:
        pass
    
    try:
        includes_range = range(len(json_response['includes']['tweets']))
        includes_id = []
        includes_retweet = []
            
        for i in includes_range:
            id_info = json_response['includes']['tweets'][i]['id']
            text = json_response['includes']['tweets'][i]['text']

            includes_id.append(id_info)
            includes_retweet.append(text)
            retweet_df = ''
            retweet_df = pd.DataFrame(list(zip(includes_id, includes_retweet)), 
                       columns =['retweet_id', 'text'])
    
    except KeyError:
        pass
        
    retweet_df_disk = pd.concat([retweet_df_disk, retweet_df], axis=0)
    retweet_df_disk = retweet_df_disk.drop_duplicates()
                
    if 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
    else:
        break

engagement = df['public_metrics'].apply(pd.Series)
df = pd.concat([df[['id', 'author_id','created_at', 'text', 'type', 'retweet_id']], engagement], axis=1)
final_data = df.merge(retweet_df_disk, on='retweet_id', how='left')
final_data['retweet_account'] = final_data.text_x.str.extract(r'(RT \@\w+)', expand = True)
final_data.text_y.fillna(final_data.text_x, inplace=True)
final_data.type.fillna("original", inplace=True)
final_data['text_z'] = final_data[['retweet_account', 'text_y']].apply(lambda x: ' '.join(x.dropna()), axis=1)
final_data = final_data[['id', 'author_id', 'created_at', 'type', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'text_z']].rename(columns={"text_z": "text"})
final_data

79


,id,author_id,created_at,type,retweet_count,reply_count,like_count,quote_count,text
0,1477769524011315210,826104766161879044,2022-01-02T22:31:24.000Z,replied_to,0,1,0,0,I didn’t realised you got tax credits for lyin...
1,1477750903025319937,1433581386112380932,2022-01-02T21:17:25.000Z,replied_to,1,0,4,0,@AccidentAlert_K @sikikasafety @road_driving @...
2,1477743788793778179,1273894620,2022-01-02T20:49:09.000Z,retweeted,3,0,0,0,RT @PRSA_Roadsafety Let be honest PSV drivers ...
3,1477740238323789831,19198704,2022-01-02T20:35:02.000Z,original,0,0,2,0,"You’ve heard of the “Great Resignation,” but a..."
4,1477737600479866884,948674925950328833,2022-01-02T20:24:33.000Z,replied_to,1,0,4,0,@AccidentAlert_K @sikikasafety @road_driving @...
...,...,...,...,...,...,...,...,...,...
74,1477119968584126464,18586040,2022-01-01T03:30:18.000Z,original,0,0,0,0,[USA] Public Relations Sr. Advisor for Infrast...
75,1477100493830864903,436045478,2022-01-01T02:12:55.000Z,retweeted,1,0,0,0,RT @DWJr16 It has been my honor to serve as 20...
76,1477098881880055814,1049005198860009473,2022-01-01T02:06:31.000Z,original,1,0,4,0,It has been my honor to serve as 2021 @PRSAHea...
77,1477095571777155073,9745402,2022-01-01T01:53:22.000Z,retweeted,5,0,0,0,"RT @PRSA As we close out 2021, we want to take..."


## Importing User Name from id

In [5]:
client = Twarc2(bearer_token=TOKEN)
user_id_list = final_data["author_id"]
lookup = client.user_lookup(users=user_id_list)

username = []
follower_n = []
following_n = []

for page in lookup:
    result = expansions.flatten(page)
    for user in result:
        username.append(user['username'])
        follower_n.append(user['public_metrics']['followers_count'])
        following_n.append(user['public_metrics']['following_count'])
        
final_data["username"] = username
final_data["follower_count"] = follower_n
final_data["following_count"] = following_n
final_data

,id,author_id,created_at,type,retweet_count,reply_count,like_count,quote_count,text,username,follower_count,following_count
0,1477769524011315210,826104766161879044,2022-01-02T22:31:24.000Z,replied_to,0,1,0,0,I didn’t realised you got tax credits for lyin...,_postpunkdad,687,430
1,1477750903025319937,1433581386112380932,2022-01-02T21:17:25.000Z,replied_to,1,0,4,0,@AccidentAlert_K @sikikasafety @road_driving @...,Missy41091460,3,29
2,1477743788793778179,1273894620,2022-01-02T20:49:09.000Z,retweeted,3,0,0,0,RT @PRSA_Roadsafety Let be honest PSV drivers ...,kiela_w,980,798
3,1477740238323789831,19198704,2022-01-02T20:35:02.000Z,original,0,0,2,0,"You’ve heard of the “Great Resignation,” but a...",PRSASacramento,2374,1497
4,1477737600479866884,948674925950328833,2022-01-02T20:24:33.000Z,replied_to,1,0,4,0,@AccidentAlert_K @sikikasafety @road_driving @...,EliasZakayo300,443,1304
...,...,...,...,...,...,...,...,...,...,...,...,...
74,1477119968584126464,18586040,2022-01-01T03:30:18.000Z,original,0,0,0,0,[USA] Public Relations Sr. Advisor for Infrast...,pr_jobs,5746,3255
75,1477100493830864903,436045478,2022-01-01T02:12:55.000Z,retweeted,1,0,0,0,RT @DWJr16 It has been my honor to serve as 20...,PRSAHealth,2229,1543
76,1477098881880055814,1049005198860009473,2022-01-01T02:06:31.000Z,original,1,0,4,0,It has been my honor to serve as 2021 @PRSAHea...,DWJr16,195,1338
77,1477095571777155073,9745402,2022-01-01T01:53:22.000Z,retweeted,5,0,0,0,"RT @PRSA As we close out 2021, we want to take...",alaskaegan,507,458


## Export the data as a excel file

In [6]:
final_data.to_excel("Twitter_data.xlsx")

## The most-retweeted content (top 10)

In [7]:
final_data.drop_duplicates(subset = ["text"]).sort_values(by=['retweet_count'], ascending=False)[0:10]

,id,author_id,created_at,type,retweet_count,reply_count,like_count,quote_count,text,username,follower_count,following_count
11,1477677744511201282,424707168,2022-01-02T16:26:42.000Z,retweeted,11,0,0,0,RT @PRSA Happy New Year! Looking forward to a ...,othatnikkiyoung,1786,2832
10,1477679205945012227,4042432272,2022-01-02T16:32:31.000Z,retweeted,5,0,0,0,"RT @PRSA As we close out 2021, we want to take...",PRSAGoldCoast,177,154
47,1477383860627386368,350430192,2022-01-01T20:58:55.000Z,retweeted,5,0,0,0,RT @NHSLincsCCG We've opened up more vaccinati...,ANPCatherine,184,855
41,1477468387685113856,705395511155695616,2022-01-02T02:34:48.000Z,original,4,2,14,1,Happy New Year! Is earning the APR credential ...,PRaccred,1199,304
38,1477507276437135362,299123939,2022-01-02T05:09:20.000Z,retweeted,4,0,0,0,RT @PRaccred Happy New Year! Is earning the AP...,adriennefoley66,156,657
2,1477743788793778179,1273894620,2022-01-02T20:49:09.000Z,retweeted,3,0,0,0,RT @PRSA_Roadsafety Let be honest PSV drivers ...,kiela_w,980,798
5,1477723373279064069,270478924,2022-01-02T19:28:01.000Z,retweeted,2,0,0,0,"RT @PRSA_Roadsafety Chaotic, Madness , crazine...",JesseKenya,4252,1122
50,1477352252692209670,25765161,2022-01-01T18:53:19.000Z,retweeted,2,0,0,0,RT @prsa_pag #HappyNewYear to our #PAG members...,ufcommunigator,584,1050
69,1477163109001351168,2524077089,2022-01-01T06:21:44.000Z,retweeted,1,0,0,0,RT @pr_jobs [USA] Public Relations (PR) Specia...,TorivoJobs,3335,2237
68,1477191460781015044,156050161,2022-01-01T08:14:23.000Z,retweeted,1,0,0,0,RT @PRSA_SA Hello 2️⃣0️⃣2️⃣2️⃣ 💫🥂 Wishing ever...,UTCheadle,706,446


## Twitter users who wrote tweets the most about the issue

In [8]:
talktive = final_data['username'].value_counts().reset_index()[0:10].rename({"index": "username", "username": "tweet n"}, axis = 1) 
talktive

,username,tweet n
0,pr_jobs,5
1,TorivoJobs,3
2,UMDearbornPRSSA,2
3,doctrahanmedia,2
4,professordavem,2
5,PRSAHealth,2
6,DWJr16,2
7,careersingov,2
8,usakenshin,2
9,PRSAmadison,1


## Twitter users with the top 10 highest followers (top 10)

In [9]:
final_data.sort_values(by=['follower_count'], ascending=False).drop_duplicates(subset = ["username"])[0:10]

,id,author_id,created_at,type,retweet_count,reply_count,like_count,quote_count,text,username,follower_count,following_count
65,1477305116378210308,14804281,2022-01-01T15:46:01.000Z,original,11,0,33,4,Happy New Year! Looking forward to a brighter ...,PRSA,107071,622
45,1477435979900272640,525005120,2022-01-02T00:26:01.000Z,original,0,0,0,0,Amazing #careeropportunity with @ValleyWater. ...,careersingov,100236,87262
8,1477715105127321604,16463639,2022-01-02T18:55:10.000Z,retweeted,2,0,0,0,"RT @PRSA_Roadsafety Chaotic, Madness , crazine...",Mwirigi,73089,7962
17,1477639295485853696,385325759,2022-01-02T13:53:55.000Z,original,1,0,2,1,Many feared dead this morning at Musembe along...,papafavour,13065,9917
31,1477543259983417344,2381875884,2022-01-02T07:32:19.000Z,retweeted,1,0,0,0,RT @PRSA_Roadsafety @ntsa_kenya @IG_NPS @NPSC_...,julishwa,6898,4223
14,1477655552629084160,1443305123879530506,2022-01-02T14:58:31.000Z,retweeted,1,0,0,0,RT @usakenshin @Prsa_samurai @nazanin_samurai ...,Okami_mxsamurai,6614,2824
37,1477510025765609475,18586040,2022-01-02T05:20:15.000Z,original,1,0,0,0,[USA] Public Relations Specialist - ICF - Arli...,pr_jobs,5746,3255
6,1477716100003553286,40591485,2022-01-02T18:59:07.000Z,retweeted,1,0,0,0,RT @careersingov .@ValleyMetro is #hiring a MA...,HiringCare,4323,573
5,1477723373279064069,270478924,2022-01-02T19:28:01.000Z,retweeted,2,0,0,0,"RT @PRSA_Roadsafety Chaotic, Madness , crazine...",JesseKenya,4252,1122
42,1477464169767133192,41019284,2022-01-02T02:18:02.000Z,quoted,0,0,3,0,Happy New Year! Looking forward to a brighter ...,PRSADetroit,3918,2525
